In [23]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [24]:
# Построение автокодировщика
def build_autoencoder(input_shape=(128, 128, 3)):
    input_img = Input(shape=input_shape)

    # Кодировщик
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Декодировщик
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return autoencoder


In [25]:
# Загрузка изображений из директории
def load_images_from_directory(directory, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Масштабируем изображение к нужному размеру
                img_resized = cv2.resize(img, target_size)
                img_normalized = img_resized / 255.0  # Нормализуем пиксели
                images.append(img_normalized)
    images = np.array(images)
    return images

In [26]:
# Класс детектора аномалий
class AnomalyDetector:
    def __init__(self, model_path, crop_region, input_size=(128, 128), threshold=0.01):
        self.model = tf.keras.models.load_model(model_path)
        # Повторно компилируем модель, если планируем её использовать не только для предсказаний
        self.model.compile(optimizer='adam', loss='binary_crossentropy')
        
        self.crop_region = crop_region  # Регион для обрезки
        self.input_size = input_size  # Размер изображений для подачи в модель
        self.threshold = threshold  # Пороговое значение для обнаружения аномалий

    def crop_image(self, img):
        """Обрезка изображения до региона интереса."""
        x, y, w, h = self.crop_region
        return img[y:y+h, x:x+w]

    def preprocess_image(self, img):
        """Предобработка изображения для подачи в автокодировщик."""
        cropped_img = self.crop_image(img)
        img_resized = cv2.resize(cropped_img, self.input_size)  # Приводим к нужному размеру
        img_normalized = img_resized / 255.0  # Нормализуем пиксели в диапазон [0, 1]
        return np.expand_dims(img_normalized, axis=0)

    def is_anomaly(self, img):
        """Проверка изображения на наличие аномалий."""
        preprocessed_img = self.preprocess_image(img)
        reconstructed_img = self.model.predict(preprocessed_img)
        loss = np.mean(np.abs(reconstructed_img - preprocessed_img))  # Вычисляем разницу между входом и выходом
        return loss > self.threshold

    def predict_on_image(self, img_path):
        """Загрузка изображения и проверка на аномалии."""
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f"Не удалось загрузить изображение: {img_path}")

        if self.is_anomaly(img):
            print(f"Аномалия обнаружена на изображении: {img_path}")
        else:
            print(f"Все в порядке на изображении: {img_path}")

    def predict_on_directory(self, source_dir):
        """Проверка всех изображений в директории на наличие аномалий."""
        for filename in os.listdir(source_dir):
            if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
                img_path = os.path.join(source_dir, filename)
                self.predict_on_image(img_path)

In [27]:
if __name__ == "__main__":
    # Обучение автокодировщика
    # Путь к обрезанным изображениям
    data_dir = "object3_cut"
    
    # Задаем параметры
    input_shape = (128, 128, 3)  # Размеры изображений
    batch_size = 32
    epochs = 50

    # Загрузка всех изображений из директории
    images = load_images_from_directory(data_dir, target_size=input_shape[:2])

    # Если нужно разделить на батчи вручную
    steps_per_epoch = len(images) // batch_size

    # Преобразование данных в нужную форму для Keras
    images = np.reshape(images, (-1, input_shape[0], input_shape[1], input_shape[2]))

    # Создаем модель автокодировщика
    autoencoder = build_autoencoder(input_shape=input_shape)

    # Обучение автокодировщика
    autoencoder.fit(
        images,  # Данные на вход
        images,  # Данные на выход, так как это автокодировщик
        batch_size=batch_size,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch
    )


   

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 629ms/step - loss: 0.6736
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6169  
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 604ms/step - loss: 0.6137
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6014  
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 556ms/step - loss: 0.5918
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5542  
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 531ms/step - loss: 0.5534
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5353  
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 572ms/step - loss: 0.5155
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5181  
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 585ms/step - loss: 0.4809
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4694  
Epoch 13/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 578ms/step - loss: 0.4471
Epoch 14/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4564  
Epoch 15/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 314ms/step - loss: 0.4306
Epoch 16/50
4/4 ━━━

In [28]:
    # Сохраняем обученную модель
    autoencoder.save('anomaly_detector_autoencoder.h5')

In [38]:
# Основной код
if __name__ == "__main__":
    # Определяем регион обрезки: (x, y, width, height)
    crop_region = (1390, 180, 624, 512)

    # Создаем экземпляр класса детектора аномалий
    anomaly_detector = AnomalyDetector(
        model_path="anomaly_detector_autoencoder.h5",  # Путь к вашей модели
        crop_region=crop_region,  # Регион для обрезки изображений
        input_size=(128, 128),  # Размер входных изображений для модели
        threshold=0.04  # Порог для аномалий
    )
    
    # Проверяем изображения в папке на наличие аномалий
    source_dir = "object3_bad"
    anomaly_detector.predict_on_directory(source_dir)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 752ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.20.07.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.29.39.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.30.36.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.32.31.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.35.23.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_11.37.17.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_12.28.00.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Аномалия обнаружена на изображении: object3_bad\21.08.2024_12.51.52.png
